In [1]:
using CSV, DataFrames, Statistics

In [2]:
prefix = "aistats-rev/"

"aistats-rev/"

In [3]:
function method_category(meth)
    if startswith(meth, "Imp-then-Reg")
        return "Imp-then-Reg"
    elseif startswith(meth, "Joint Imp-then-Reg")
        return "Joint Imp-then-Reg"
    elseif meth ∈ ["Static", "Affine", "Finite"]
        return "Adaptive LR"
    elseif startswith(meth, "Complete Features")
        return "Complete Features"
    else 
        return meth
    end
end

method_category (generic function with 1 method)

In [4]:
pb_datasets = ["cylinder-bands", "ozone-level-detection-eight", "ozone-level-detection-one", "thyroid-disease-thyroid-0387", "trains",
                "credit-approval", "Ecdat-Mofa", "sleep"]

8-element Vector{String}:
 "cylinder-bands"
 "ozone-level-detection-eight"
 "ozone-level-detection-one"
 "thyroid-disease-thyroid-0387"
 "trains"
 "credit-approval"
 "Ecdat-Mofa"
 "sleep"

## For Real X - Syn Y Experiments

In [5]:
setting = prefix*"fakey/"
for y_model in ["linear", "nn"]
    for m_model = ["mar", "nmar", "mar_adv"]
        dir = y_model*"_"*m_model*"/"
        directory = setting*dir
        
        filelist = [f for f in readdir(directory*"all/") if endswith(f, ".csv")]
        res = similar(CSV.read(directory*"all/"*filelist[1], DataFrame),0)
        
        for subdir = ["all/"]
            filelist = [f for f in readdir(directory*subdir) if endswith(f, ".csv")]
#             res = similar(CSV.read(directory*subdir*filelist[1], DataFrame),0)
            for i in 1:length(filelist)
                res = vcat(res, CSV.read(directory*subdir*filelist[i], DataFrame))
            end
        end

    #     for subdir = ["xgb/"]
    #         try 
    #             filelist = [f for f in readdir("xgboost/"*directory*subdir) if endswith(f, ".csv")]
    #             for i in 1:length(filelist)
    #                 res = vcat(res, CSV.read("xgboost/"*directory*subdir*filelist[i], DataFrame))
    #             end
    #         catch 
    #             println("No XGBoost results for "*directory*subdir)
    #         end
    #     end

        res[!,:method_cat] = map(t -> method_category(t), res[:,:method])
        res[!,:X_setting] .= "real_X_"*m_model
        res[!,:Y_setting] .= "syn_Y_"*y_model

        CSV.write(directory*"FINAL_results.csv", res)
    end
end

In [6]:
setting = prefix*"fakey/"

for y_model in ["linear", "nn"]
    for m_model = ["mar", "nmar", "mar_adv"]
        dir = y_model*"_"*m_model*"/"
        directory = setting*dir
  
        res = CSV.read(directory*"FINAL_results.csv", DataFrame)
        
        res[!,:method] .= map(t -> (t == "Affine" ? "Adaptive LR - Affine" : t), res[:,:method]) 
        res[!,:method] .= map(t -> (t == "Finite" ? "Adaptive LR - Finite" : t), res[:,:method]) 
        res[!,:method] .= map(t -> (t == "Static" ? "Adaptive LR - Affine intercept only" : t), res[:,:method])
        
        filter!(t -> t[:dataset] ∉ pb_datasets, res)

        for method in ["Oracle X", "Oracle XM", "Complete Features", "Imp-then-Reg 1", "Imp-then-Reg 2", "Imp-then-Reg 3", "Imp-then-Reg 4", "Imp-then-Reg 5", "Joint Imp-then-Reg", "Adaptive LR"]
            aux = filter(t -> startswith(t[:method], method), res)

            idcols = [:dataset, :X_setting, :Y_setting, :SNR, :k, :kMissing, :splitnum]
            gd = groupby(aux, idcols)

            aux = similar(aux, 0)
            for subdf in gd 
                scoremax = argmax(subdf[:,:score])
                push!(aux, subdf[scoremax,names(aux)])
            end
            aux[!,:method] .= method*" - best"

            res = vcat(res, aux)
        end
        
        CSV.write(directory*"FINAL_results.csv", res)
    end
end

In [25]:
for y_model in ["linear", "nn"]
    for m_model = ["mar", "nmar", "mar_adv"]
        dir = y_model*"_"*m_model*"/"
        directory = prefix*"fakey/"*dir
  
        res = CSV.read(directory*"FINAL_results.csv", DataFrame)
        
        unfinished_datasets = filter(t -> t[:nrow] < 10, combine(groupby( combine(groupby(res, [:dataset, :method]), nrow), [:dataset]), :nrow => maximum => :nrow))[:,:dataset]

        @show unfinished_datasets
    end
end

unfinished_datasets = ["Ecdat-Males", "Ecdat-Schooling", "mushroom", "thyroid-disease-allbp", "thyroid-disease-allhyper", "thyroid-disease-allhypo", "thyroid-disease-allrep", "thyroid-disease-dis", "thyroid-disease-sick-euthyroid", "thyroid-disease-sick", "wiki4he"]


unfinished_datasets = ["Ecdat-Males", "Ecdat-RetSchool", "Ecdat-Schooling", "arrhythmia", "car-SLID", "mushroom", "thyroid-disease-allbp", "thyroid-disease-allhypo", "thyroid-disease-allrep", "thyroid-disease-sick-euthyroid", "thyroid-disease-sick", "wiki4he"]


unfinished_datasets = ["Ecdat-Males", "Ecdat-RetSchool", "Ecdat-Schooling", "car-SLID", "mlmRev-Gcsemv", "robustbase-ambientNOxCH", "thyroid-disease-allbp", "thyroid-disease-allhyper", "thyroid-disease-allhypo", "thyroid-disease-allrep", "thyroid-disease-dis", "thyroid-disease-sick-euthyroid", "thyroid-disease-sick"]


unfinished_datasets = ["Ecdat-Males", "Ecdat-RetSchool", "Ecdat-Schooling", "thyroid-disease-allbp", "thyroid-disease-allhyper", "thyroid-disease-allhypo", "thyroid-disease-allrep", "thyroid-disease-dis", "thyroid-disease-sick-euthyroid", "thyroid-disease-sick"]


unfinished_datasets = ["Ecdat-Males", "Ecdat-Schooling", "arrhythmia", "car-SLID", "mlmRev-Gcsemv", "pscl-politicalInformation", "thyroid-disease-allhyper", "thyroid-disease-allhypo", "thyroid-disease-allrep", "thyroid-disease-dis", "thyroid-disease-sick-euthyroid", "thyroid-disease-sick", "wiki4he"]


unfinished_datasets = ["Ecdat-Males", "Ecdat-RetSchool", "Ecdat-Schooling", "arrhythmia", "car-SLID", "communities-and-crime", "thyroid-disease-allbp", "thyroid-disease-allhyper", "thyroid-disease-allhypo", "thyroid-disease-allrep", "thyroid-disease-dis", "thyroid-disease-sick"]


## For Real Data Experiments

In [7]:
1+1

2

In [8]:
for directory = prefix .* ["realy/"]
    filelist = [f for f in readdir(directory*"all/") if endswith(f, ".csv") && f ∉ ["all_results.csv","all_results_new.csv"]]
    res = similar(CSV.read(directory*"all/"*filelist[1], DataFrame),0)
    
    # for subdir = ["2022-08-23/", "rf_mia/"]    
    for subdir = ["all/"]
        filelist = [f for f in readdir(directory*subdir) if endswith(f, ".csv") && f ∉ ["all_results.csv","all_results_new.csv"]]
#         res = similar(CSV.read(directory*subdir*filelist[1], DataFrame),0)
        for i in 1:length(filelist)
            res = vcat(res, CSV.read(directory*subdir*filelist[i], DataFrame))
        end
    end

#     for subdir = ["xgb/"]
#         try 
#             filelist = [f for f in readdir("xgboost/"*directory*subdir) if endswith(f, ".csv")]
#             for i in 1:length(filelist)
#                 res = vcat(res, CSV.read("xgboost/"*directory*subdir*filelist[i], DataFrame))
#             end
#         catch 
#             println("No XGBoost results for "*directory*subdir)
#         end
#     end
    
    # filter!(t -> t[:k] > 0, res) #Remove dataset with only a bias term
    res[!,:method_cat] = map(method_category, res[:,:method])
    res[!,:X_setting] .= "real_X"
    res[!,:Y_setting] .= "real_Y"
    
    CSV.write(directory*"FINAL_results.csv", res)
end

Sanity check

In [9]:
res = CSV.read(prefix*"realy/"*"FINAL_results.csv", DataFrame)

unique(filter( t-> t[:nrow] < 10, combine(groupby(res, [:dataset, :method]), nrow))[:,[:dataset, :nrow]])

Row,dataset,nrow
,String,Int64
1,Ecdat-Males,7
2,Ecdat-RetSchool,7
3,Ecdat-Schooling,3
4,arrhythmia,3
5,breast-cancer-wisconsin-prognostic,9
6,communities-and-crime-2,2
7,communities-and-crime,3
8,credit-approval,9
9,heart-disease-va,9


In [14]:
dataset_list = [d for d in readdir("../datasets/") if !startswith(d, ".")]
sort!(dataset_list)

71-element Vector{String}:
 "COUNT-loomis"
 "Ecdat-MCAS"
 "Ecdat-Males"
 "Ecdat-Mofa"
 "Ecdat-RetSchool"
 "Ecdat-Schooling"
 "MASS-Cars93"
 "MASS-Pima.tr2"
 "MASS-survey"
 "Zelig-coalition2"
 ⋮
 "thyroid-disease-allhyper"
 "thyroid-disease-allhypo"
 "thyroid-disease-allrep"
 "thyroid-disease-dis"
 "thyroid-disease-sick"
 "thyroid-disease-sick-euthyroid"
 "thyroid-disease-thyroid-0387"
 "trains"
 "wiki4he"

In [23]:
unfinished_datasets = filter(t -> t[:nrow] < 10, combine(groupby( combine(groupby(res, [:dataset, :method]), nrow), [:dataset]), :nrow => maximum => :nrow))[:,:dataset]
[findfirst(i .== dataset_list)-1 for i in unfinished_datasets]

15-element Vector{Int64}:
  2
  4
  5
 10
 16
 24
 23
 26
 36
 38
 41
 44
 49
 64
 68

In [13]:
setdiff(), pb_datasets)

13-element Vector{String}:
 "Ecdat-Males"
 "Ecdat-RetSchool"
 "Ecdat-Schooling"
 "arrhythmia"
 "breast-cancer-wisconsin-prognostic"
 "communities-and-crime-2"
 "communities-and-crime"
 "heart-disease-va"
 "horse-colic"
 "mammographic-mass"
 "mushroom"
 "pscl-politicalInformation"
 "thyroid-disease-allrep"

Create `best` variant

In [24]:
res = CSV.read(prefix*"realy/"*"FINAL_results.csv", DataFrame)

filter!(t -> t[:dataset] ∉ pb_datasets, res)

res[!,:method] .= map(t -> (t == "Affine" ? "Adaptive LR - Affine" : t), res[:,:method]) 
res[!,:method] .= map(t -> (t == "Finite" ? "Adaptive LR - Finite" : t), res[:,:method]) 
res[!,:method] .= map(t -> (t == "Static" ? "Adaptive LR - Affine intercept only" : t), res[:,:method])

for method in ["Complete Features", "Imp-then-Reg 1", "Imp-then-Reg 2", "Imp-then-Reg 3", "Imp-then-Reg 4", "Imp-then-Reg 5", "Joint Imp-then-Reg", "Adaptive LR"]
    aux = filter(t -> startswith(t[:method], method), res)
    # @show size(aux)
    idcols = [:dataset, :SNR, :k, :kMissing, :splitnum]
    gd = groupby(aux, idcols)

    aux = similar(aux, 0)
    for subdf in gd 
        scoremax = argmax(subdf[:,:score])
        push!(aux, subdf[scoremax,names(aux)])
    end
    aux[!,:method] .= method*" - best"

    res = vcat(res, aux)
end

CSV.write(prefix*"realy/"*"FINAL_results.csv", res)

"aistats-rev/realy/FINAL_results.csv"

# For Synthetic-Data Experiments

In [ ]:
1+1

In [26]:
setting = prefix*"synthetic_discrete/"
# for y_model in ["linear", "tree", "nn"]
for y_model in ["linear", "nn"]
    for m_model = ["mar", "censoring"]
        dir = y_model*"_"*m_model*"/"
        directory = setting*dir
        
        filelist = [f for f in readdir(directory*"all/") if endswith(f, ".csv") && f ∉ ["all_results.csv","all_results_new.csv"]]
        res = similar(CSV.read(directory*"all/"*filelist[1], DataFrame),0)

        for subdir = ["all/"]
            filelist = [f for f in readdir(directory*subdir) if endswith(f, ".csv") && f ∉ ["all_results.csv","all_results_new.csv"]]
            @show filelist

    #         res = similar(CSV.read(directory*subdir*filelist[1], DataFrame),0)
            for i in 1:length(filelist)
                aux = CSV.read(directory*subdir*filelist[i], DataFrame)
                # if any(aux[:,:kMissing] .> 0)
                #     missingproba = unique(aux[aux[:,:pMissing] .> 0,:pMissing])[1]
                #     aux[!,:pMissing] .= missingproba
                # end
                try
                    res = vcat(res, aux)
                catch 
                    println("Error with ", directory*subdir*filelist[i])
                end
            end
        end

        # for subdir = ["xgb/"]
        #     try 
        #         filelist = [f for f in readdir("xgboost/"*directory*subdir) if endswith(f, ".csv")]
        #         for i in 1:length(filelist)
        #             res = vcat(res, CSV.read("xgboost/"*directory*subdir*filelist[i], DataFrame))
        #         end
        #     catch 
        #         println("No XGBoost results for "*directory*subdir)
        #     end
        # end

        res[!,:method] .= map(t -> replace(t, "Encoding as new category" => "Imp-then-Reg 4"), res[:,:method])
        res[!,:method] .= map(t -> replace(t, "Mode impute" => "Imp-then-Reg 5"), res[:,:method])

        # filter!(t -> t[:k] > 0, res) #Remove dataset with only a bias term
        res[!,:method_cat] = map(method_category, res[:,:method])
        res[!,:X_setting] .= "syn_X_"*m_model
        res[!,:Y_setting] .= "syn_Y_"*y_model

        for method in ["Imp-then-Reg 4", "Imp-then-Reg 5"]
            aux = filter(t -> startswith(t[:method], method), res)
            # @show nrow(aux)
            idcols = [:dataset, :X_setting, :Y_setting, :SNR, :k, :pMissing, :splitnum]
            gd = groupby(aux, idcols)

            aux = similar(aux, 0)
            for subdf in gd 
                scoremax = argmax(subdf[:,:score])
                # @show subdf[scoremax,names(aux)]
                push!(aux, subdf[scoremax,names(aux)])
            end
            aux[!,:method] .= method*" - best"

            res = vcat(res, aux)
        end

        CSV.write(directory*"FINAL_results.csv", res)   
    end
end

filelist = ["n_1000_p_10_pmiss_0.1_1.csv", "n_1000_p_10_pmiss_0.1_10.csv", "n_1000_p_10_pmiss_0.1_2.csv", "n_1000_p_10_pmiss_0.1_3.csv", "n_1000_p_10_pmiss_0.1_4.csv", "n_1000_p_10_pmiss_0.1_5.csv", "n_1000_p_10_pmiss_0.1_6.csv", "n_1000_p_10_pmiss_0.1_7.csv", "n_1000_p_10_pmiss_0.1_8.csv", "n_1000_p_10_pmiss_0.1_9.csv", "n_1000_p_10_pmiss_0.2_1.csv", "n_1000_p_10_pmiss_0.2_10.csv", "n_1000_p_10_pmiss_0.2_2.csv", "n_1000_p_10_pmiss_0.2_3.csv", "n_1000_p_10_pmiss_0.2_4.csv", "n_1000_p_10_pmiss_0.2_5.csv", "n_1000_p_10_pmiss_0.2_6.csv", "n_1000_p_10_pmiss_0.2_7.csv", "n_1000_p_10_pmiss_0.2_8.csv", "n_1000_p_10_pmiss_0.2_9.csv", "n_1000_p_10_pmiss_0.3_1.csv", "n_1000_p_10_pmiss_0.3_10.csv", "n_1000_p_10_pmiss_0.3_2.csv", "n_1000_p_10_pmiss_0.3_3.csv", "n_1000_p_10_pmiss_0.3_4.csv", "n_1000_p_10_pmiss_0.3_5.csv", "n_1000_p_10_pmiss_0.3_6.csv", "n_1000_p_10_pmiss_0.3_7.csv", "n_1000_p_10_pmiss_0.3_8.csv", "n_1000_p_10_pmiss_0.3_9.csv", "n_1000_p_10_pmiss_0.4_1.csv", "n_1000_p_10_pmiss_0.4_1

", "n_580_p_10_pmiss_0.3_1.csv", "n_580_p_10_pmiss_0.3_10.csv", "n_580_p_10_pmiss_0.3_2.csv", "n_580_p_10_pmiss_0.3_3.csv", "n_580_p_10_pmiss_0.3_4.csv", "n_580_p_10_pmiss_0.3_5.csv", "n_580_p_10_pmiss_0.3_6.csv", "n_580_p_10_pmiss_0.3_7.csv", "n_580_p_10_pmiss_0.3_8.csv", "n_580_p_10_pmiss_0.3_9.csv", "n_580_p_10_pmiss_0.4_1.csv", "n_580_p_10_pmiss_0.4_10.csv", "n_580_p_10_pmiss_0.4_2.csv", "n_580_p_10_pmiss_0.4_3.csv", "n_580_p_10_pmiss_0.4_4.csv", "n_580_p_10_pmiss_0.4_5.csv", "n_580_p_10_pmiss_0.4_6.csv", "n_580_p_10_pmiss_0.4_7.csv", "n_580_p_10_pmiss_0.4_8.csv", "n_580_p_10_pmiss_0.4_9.csv", "n_580_p_10_pmiss_0.5_1.csv", "n_580_p_10_pmiss_0.5_10.csv", "n_580_p_10_pmiss_0.5_2.csv", "n_580_p_10_pmiss_0.5_3.csv", "n_580_p_10_pmiss_0.5_4.csv", "n_580_p_10_pmiss_0.5_5.csv", "n_580_p_10_pmiss_0.5_6.csv", "n_580_p_10_pmiss_0.5_7.csv", "n_580_p_10_pmiss_0.5_8.csv", "n_580_p_10_pmiss_0.5_9.csv", "n_580_p_10_pmiss_0.6_1.csv", "n_580_p_10_pmiss_0.6_10.csv", "n_580_p_10_pmiss_0.6_2.csv", "n_

filelist = ["n_1000_p_10_pmiss_0.1_1.csv", "n_1000_p_10_pmiss_0.1_10.csv", "n_1000_p_10_pmiss_0.1_2.csv", "n_1000_p_10_pmiss_0.1_3.csv", "n_1000_p_10_pmiss_0.1_4.csv", "n_1000_p_10_pmiss_0.1_5.csv", "n_1000_p_10_pmiss_0.1_6.csv", "n_1000_p_10_pmiss_0.1_7.csv", "n_1000_p_10_pmiss_0.1_8.csv", "n_1000_p_10_pmiss_0.1_9.csv", "n_1000_p_10_pmiss_0.2_1.csv", "n_1000_p_10_pmiss_0.2_10.csv", "n_1000_p_10_pmiss_0.2_2.csv", "n_1000_p_10_pmiss_0.2_3.csv", "n_1000_p_10_pmiss_0.2_4.csv", "n_1000_p_10_pmiss_0.2_5.csv", "n_1000_p_10_pmiss_0.2_6.csv", "n_1000_p_10_pmiss_0.2_7.csv", "n_1000_p_10_pmiss_0.2_8.csv", "n_1000_p_10_pmiss_0.2_9.csv", "n_1000_p_10_pmiss_0.3_1.csv", "n_1000_p_10_pmiss_0.3_10.csv", "n_1000_p_10_pmiss_0.3_2.csv", "n_1000_p_10_pmiss_0.3_3.csv", "n_1000_p_10_pmiss_0.3_4.csv", "n_1000_p_10_pmiss_0.3_5.csv", "n_1000_p_10_pmiss_0.3_6.csv", "n_1000_p_10_pmiss_0.3_7.csv", "n_1000_p_10_pmiss_0.3_8.csv", "n_1000_p_10_pmiss_0.3_9.csv", "n_1000_p_10_pmiss_0.4_1.csv", "n_1000_p_10_pmiss_0.4_1

", "n_580_p_10_pmiss_0.3_1.csv", "n_580_p_10_pmiss_0.3_10.csv", "n_580_p_10_pmiss_0.3_2.csv", "n_580_p_10_pmiss_0.3_3.csv", "n_580_p_10_pmiss_0.3_4.csv", "n_580_p_10_pmiss_0.3_5.csv", "n_580_p_10_pmiss_0.3_6.csv", "n_580_p_10_pmiss_0.3_7.csv", "n_580_p_10_pmiss_0.3_8.csv", "n_580_p_10_pmiss_0.3_9.csv", "n_580_p_10_pmiss_0.4_1.csv", "n_580_p_10_pmiss_0.4_10.csv", "n_580_p_10_pmiss_0.4_2.csv", "n_580_p_10_pmiss_0.4_3.csv", "n_580_p_10_pmiss_0.4_4.csv", "n_580_p_10_pmiss_0.4_5.csv", "n_580_p_10_pmiss_0.4_6.csv", "n_580_p_10_pmiss_0.4_7.csv", "n_580_p_10_pmiss_0.4_8.csv", "n_580_p_10_pmiss_0.4_9.csv", "n_580_p_10_pmiss_0.5_1.csv", "n_580_p_10_pmiss_0.5_10.csv", "n_580_p_10_pmiss_0.5_2.csv", "n_580_p_10_pmiss_0.5_3.csv", "n_580_p_10_pmiss_0.5_4.csv", "n_580_p_10_pmiss_0.5_5.csv", "n_580_p_10_pmiss_0.5_6.csv", "n_580_p_10_pmiss_0.5_7.csv", "n_580_p_10_pmiss_0.5_8.csv", "n_580_p_10_pmiss_0.5_9.csv", "n_580_p_10_pmiss_0.6_1.csv", "n_580_p_10_pmiss_0.6_10.csv", "n_580_p_10_pmiss_0.6_2.csv", "n_

filelist = ["n_1000_p_10_pmiss_0.1_1.csv", "n_1000_p_10_pmiss_0.1_10.csv", "n_1000_p_10_pmiss_0.1_2.csv", "n_1000_p_10_pmiss_0.1_3.csv", "n_1000_p_10_pmiss_0.1_4.csv", "n_1000_p_10_pmiss_0.1_5.csv", "n_1000_p_10_pmiss_0.1_6.csv", "n_1000_p_10_pmiss_0.1_7.csv", "n_1000_p_10_pmiss_0.1_8.csv", "n_1000_p_10_pmiss_0.1_9.csv", "n_1000_p_10_pmiss_0.2_1.csv", "n_1000_p_10_pmiss_0.2_10.csv", "n_1000_p_10_pmiss_0.2_2.csv", "n_1000_p_10_pmiss_0.2_3.csv", "n_1000_p_10_pmiss_0.2_4.csv", "n_1000_p_10_pmiss_0.2_5.csv", "n_1000_p_10_pmiss_0.2_6.csv", "n_1000_p_10_pmiss_0.2_7.csv", "n_1000_p_10_pmiss_0.2_8.csv", "n_1000_p_10_pmiss_0.2_9.csv", "n_1000_p_10_pmiss_0.3_1.csv", "n_1000_p_10_pmiss_0.3_10.csv", "n_1000_p_10_pmiss_0.3_2.csv", "n_1000_p_10_pmiss_0.3_3.csv", "n_1000_p_10_pmiss_0.3_4.csv", "n_1000_p_10_pmiss_0.3_5.csv", "n_1000_p_10_pmiss_0.3_6.csv", "n_1000_p_10_pmiss_0.3_7.csv", "n_1000_p_10_pmiss_0.3_8.csv", "n_1000_p_10_pmiss_0.3_9.csv", "n_1000_p_10_pmiss_0.4_1.csv", "n_1000_p_10_pmiss_0.4_1

", "n_580_p_10_pmiss_0.3_1.csv", "n_580_p_10_pmiss_0.3_10.csv", "n_580_p_10_pmiss_0.3_2.csv", "n_580_p_10_pmiss_0.3_3.csv", "n_580_p_10_pmiss_0.3_4.csv", "n_580_p_10_pmiss_0.3_5.csv", "n_580_p_10_pmiss_0.3_6.csv", "n_580_p_10_pmiss_0.3_7.csv", "n_580_p_10_pmiss_0.3_8.csv", "n_580_p_10_pmiss_0.3_9.csv", "n_580_p_10_pmiss_0.4_1.csv", "n_580_p_10_pmiss_0.4_10.csv", "n_580_p_10_pmiss_0.4_2.csv", "n_580_p_10_pmiss_0.4_3.csv", "n_580_p_10_pmiss_0.4_4.csv", "n_580_p_10_pmiss_0.4_5.csv", "n_580_p_10_pmiss_0.4_6.csv", "n_580_p_10_pmiss_0.4_7.csv", "n_580_p_10_pmiss_0.4_8.csv", "n_580_p_10_pmiss_0.4_9.csv", "n_580_p_10_pmiss_0.5_1.csv", "n_580_p_10_pmiss_0.5_10.csv", "n_580_p_10_pmiss_0.5_2.csv", "n_580_p_10_pmiss_0.5_3.csv", "n_580_p_10_pmiss_0.5_4.csv", "n_580_p_10_pmiss_0.5_5.csv", "n_580_p_10_pmiss_0.5_6.csv", "n_580_p_10_pmiss_0.5_7.csv", "n_580_p_10_pmiss_0.5_8.csv", "n_580_p_10_pmiss_0.5_9.csv", "n_580_p_10_pmiss_0.6_1.csv", "n_580_p_10_pmiss_0.6_10.csv", "n_580_p_10_pmiss_0.6_2.csv", "n_

filelist = ["n_1000_p_10_pmiss_0.1_1.csv", "n_1000_p_10_pmiss_0.1_10.csv", "n_1000_p_10_pmiss_0.1_2.csv", "n_1000_p_10_pmiss_0.1_3.csv", "n_1000_p_10_pmiss_0.1_4.csv", "n_1000_p_10_pmiss_0.1_5.csv", "n_1000_p_10_pmiss_0.1_6.csv", "n_1000_p_10_pmiss_0.1_7.csv", "n_1000_p_10_pmiss_0.1_8.csv", "n_1000_p_10_pmiss_0.1_9.csv", "n_1000_p_10_pmiss_0.2_1.csv", "n_1000_p_10_pmiss_0.2_10.csv", "n_1000_p_10_pmiss_0.2_2.csv", "n_1000_p_10_pmiss_0.2_3.csv", "n_1000_p_10_pmiss_0.2_4.csv", "n_1000_p_10_pmiss_0.2_5.csv", "n_1000_p_10_pmiss_0.2_6.csv", "n_1000_p_10_pmiss_0.2_7.csv", "n_1000_p_10_pmiss_0.2_8.csv", "n_1000_p_10_pmiss_0.2_9.csv", "n_1000_p_10_pmiss_0.3_1.csv", "n_1000_p_10_pmiss_0.3_10.csv", "n_1000_p_10_pmiss_0.3_2.csv", "n_1000_p_10_pmiss_0.3_3.csv", "n_1000_p_10_pmiss_0.3_4.csv", "n_1000_p_10_pmiss_0.3_5.csv", "n_1000_p_10_pmiss_0.3_6.csv", "n_1000_p_10_pmiss_0.3_7.csv", "n_1000_p_10_pmiss_0.3_8.csv", "n_1000_p_10_pmiss_0.3_9.csv", "n_1000_p_10_pmiss_0.4_1.csv", "n_1000_p_10_pmiss_0.4_1

", "n_580_p_10_pmiss_0.3_1.csv", "n_580_p_10_pmiss_0.3_10.csv", "n_580_p_10_pmiss_0.3_2.csv", "n_580_p_10_pmiss_0.3_3.csv", "n_580_p_10_pmiss_0.3_4.csv", "n_580_p_10_pmiss_0.3_5.csv", "n_580_p_10_pmiss_0.3_6.csv", "n_580_p_10_pmiss_0.3_7.csv", "n_580_p_10_pmiss_0.3_8.csv", "n_580_p_10_pmiss_0.3_9.csv", "n_580_p_10_pmiss_0.4_1.csv", "n_580_p_10_pmiss_0.4_10.csv", "n_580_p_10_pmiss_0.4_2.csv", "n_580_p_10_pmiss_0.4_3.csv", "n_580_p_10_pmiss_0.4_4.csv", "n_580_p_10_pmiss_0.4_5.csv", "n_580_p_10_pmiss_0.4_6.csv", "n_580_p_10_pmiss_0.4_7.csv", "n_580_p_10_pmiss_0.4_8.csv", "n_580_p_10_pmiss_0.4_9.csv", "n_580_p_10_pmiss_0.5_1.csv", "n_580_p_10_pmiss_0.5_10.csv", "n_580_p_10_pmiss_0.5_2.csv", "n_580_p_10_pmiss_0.5_3.csv", "n_580_p_10_pmiss_0.5_4.csv", "n_580_p_10_pmiss_0.5_5.csv", "n_580_p_10_pmiss_0.5_6.csv", "n_580_p_10_pmiss_0.5_7.csv", "n_580_p_10_pmiss_0.5_8.csv", "n_580_p_10_pmiss_0.5_9.csv", "n_580_p_10_pmiss_0.6_1.csv", "n_580_p_10_pmiss_0.6_10.csv", "n_580_p_10_pmiss_0.6_2.csv", "n_

In [27]:
setting = prefix*"synthetic/"
files_with_issues = []
# for y_model in ["linear", "tree", "nn"]
for y_model in ["linear", "nn"]
    for m_model = ["mar", "censoring"]
        dir = y_model*"_"*m_model*"/"
        directory = setting*dir
        filelist = [f for f in readdir(directory*"all/") if endswith(f, ".csv") && f ∉ ["all_results.csv","all_results_new.csv"]]
        res = similar(CSV.read(directory*"all/"*filelist[1], DataFrame),0)
        for subdir = ["all/"]
            filelist = [f for f in readdir(directory*subdir) if endswith(f, ".csv") && f ∉ ["all_results.csv","all_results_new.csv"]]
    #         res = similar(CSV.read(directory*subdir*filelist[1], DataFrame),0)
            for i in 1:length(filelist)
                try
                    aux = CSV.read(directory*subdir*filelist[i], DataFrame)
                    if any(aux[:,:pMissing] .> 0)
                        missingproba = unique(aux[aux[:,:pMissing] .> 0,:pMissing])[1]
                        aux[!,:pMissing] .= missingproba
                    end
                    res = vcat(res, aux)
                catch 
                    println("Error with ", directory*subdir*filelist[i])
                    push!(files_with_issues, directory*subdir*filelist[i])
                end
            end
        end

        # for subdir = ["xgb/"]
        #     try 
        #         filelist = [f for f in readdir("xgboost/"*directory*subdir) if endswith(f, ".csv")]
        #         for i in 1:length(filelist)
        #             res = vcat(res, CSV.read("xgboost/"*directory*subdir*filelist[i], DataFrame))
        #         end
        #     catch 
        #         println("No XGBoost results for "*directory*subdir)
        #     end
        # end

        # filter!(t -> t[:k] > 0, res) #Remove dataset with only a bias term
        res[!,:method_cat] = map(method_category, res[:,:method])
        res[!,:X_setting] .= "syn_X_"*m_model
        res[!,:Y_setting] .= "syn_Y_"*y_model

        for method in ["Oracle X", "Oracle XM", "Complete Features", "Imp-then-Reg 1", "Imp-then-Reg 2", "Imp-then-Reg 3", "Imp-then-Reg 4", "Imp-then-Reg 5", "Joint Imp-then-Reg", "Adaptive LR"]
            aux = filter(t -> startswith(t[:method], method), res)

            idcols = [:dataset, :X_setting, :Y_setting, :SNR, :k, :pMissing, :splitnum]
            gd = groupby(aux, idcols)

            aux = similar(aux, 0)
            for subdf in gd 
                scoremax = argmax(subdf[:,:score])
                push!(aux, subdf[scoremax,names(aux)])
            end
            aux[!,:method] .= method*" - best"

            res = vcat(res, aux)
        end
        
        CSV.write(directory*"FINAL_results.csv", res)   
    end
end

In [ ]:
files_with_issues

In [ ]:
# setting = "synthetic/"
# # for y_model in ["linear", "tree", "nn"]
# for y_model in ["linear", "nn"]
#     for m_model = ["mar", "censoring"]
#         dir = y_model*"_"*m_model*"/"
#         directory = setting*dir
        
#         filelist = [f for f in readdir(directory*"final/") if endswith(f, ".csv") && f ∉ ["all_results.csv","all_results_new.csv"]]
#         res = similar(CSV.read(directory*"final/"*filelist[1], DataFrame),0)

#         for subdir = ["high_n/"]
#             filelist = [f for f in readdir(directory*subdir) if endswith(f, ".csv") && f ∉ ["all_results.csv","all_results_new.csv"]]
#     #         res = similar(CSV.read(directory*subdir*filelist[1], DataFrame),0)
#             for i in 1:length(filelist)
#                 aux = CSV.read(directory*subdir*filelist[i], DataFrame)
#                 if any(aux[:,:pMissing] .> 0)
#                     missingproba = unique(aux[aux[:,:pMissing] .> 0,:pMissing])[1]
#                     aux[!,:pMissing] .= missingproba
#                 end
#                 try
#                     res = vcat(res, aux)
#                 catch 
#                     println("Error with ", directory*subdir*filelist[i])
#                 end
#             end
#         end
#         # filter!(t -> t[:k] > 0, res) #Remove dataset with only a bias term
#         res[!,:method_cat] = map(method_category, res[:,:method])
#         res[!,:X_setting] .= "syn_X_"*m_model
#         res[!,:Y_setting] .= "syn_Y_"*y_model

#         CSV.write(directory*"HIGHN_results.csv", res)   
#     end
# end

In [27]:
1+1

2

Sanity check

In [39]:
df = CSV.read(prefix*"synthetic/linear_mar/FINAL_results.csv", DataFrame) 
df[!,:setting] .= "1 - Lin-MAR"

aux = CSV.read(prefix*"synthetic/linear_censoring/FINAL_results.csv", DataFrame) 
aux[!,:setting] .= "2 - Lin-NMAR"
df = vcat(df, aux)
 
# aux = CSV.read(prefix*"synthetic/tree_mar/FINAL_results.csv", DataFrame) 
# aux[!,:setting] .= "3 - Tree-MAR"
# df = vcat(df, aux)

# aux = CSV.read(prefix*"synthetic/tree_censoring/FINAL_results.csv", DataFrame) 
# aux[!,:setting] .= "4 - Tree-NMAR"
# df = vcat(df, aux)

aux = CSV.read(prefix*"synthetic/nn_mar/FINAL_results.csv", DataFrame) 
aux[!,:setting] .= "5 - NN-MAR"
df = vcat(df, aux)

aux = CSV.read(prefix*"synthetic/nn_censoring/FINAL_results.csv", DataFrame) 
aux[!,:setting] .= "6 - NN-NMAR"
df = vcat(df, aux)

;

In [5]:
df = CSV.read(prefix*"synthetic_discrete/linear_mar/FINAL_results.csv", DataFrame) 
df[!,:setting] .= "1 - Lin-MAR"

aux = CSV.read(prefix*"synthetic_discrete/linear_censoring/FINAL_results.csv", DataFrame) 
aux[!,:setting] .= "2 - Lin-NMAR"
df = vcat(df, aux)
 
# aux = CSV.read(prefix*"synthetic_discrete/tree_mar/FINAL_results.csv", DataFrame) 
# aux[!,:setting] .= "3 - Tree-MAR"
# df = vcat(df, aux)

# aux = CSV.read(prefix*"synthetic_discrete/tree_censoring/FINAL_results.csv", DataFrame) 
# aux[!,:setting] .= "4 - Tree-NMAR"
# df = vcat(df, aux)

aux = CSV.read(prefix*"synthetic_discrete/nn_mar/FINAL_results.csv", DataFrame) 
aux[!,:setting] .= "5 - NN-MAR"
df = vcat(df, aux)

aux = CSV.read(prefix*"synthetic_discrete/nn_censoring/FINAL_results.csv", DataFrame) 
aux[!,:setting] .= "6 - NN-NMAR"
df = vcat(df, aux)

Row,dataset,SNR,k,pMissing,splitnum,method,r2,osr2,r2list,osr2list,muvec,time,hp,score,method_cat,X_setting,Y_setting,setting
,String15,Int64,Int64,Float64,Int64,String31,Float64,Float64,String31,String31,String7,Float64,String,Float64,String,String15,String15,String
1,n_1000_p_10,2,5,0.1,1,Oracle X,0.664324,0.620391,Any[0.6643244100492505],Any[0.620391048897614],Any[],14.0286,"Dict{Symbol, Any}(:alpha => 1.0, :regtype => :lasso)",0.660321,Oracle X,syn_X_mar,syn_Y_linear,1 - Lin-MAR
2,n_1000_p_10,2,5,0.1,1,Oracle XM,0.664324,0.620391,Any[0.6643244100492505],Any[0.620391048897614],Any[],0.16225,"Dict{Symbol, Any}(:alpha => 1.0, :regtype => :lasso)",0.660321,Oracle XM,syn_X_mar,syn_Y_linear,1 - Lin-MAR
3,n_1000_p_10,2,5,0.1,1,Complete Features,0.0,0.0,Any[],Any[],Any[],0.0,0.0,0.0,Complete Features,syn_X_mar,syn_Y_linear,1 - Lin-MAR
4,n_1000_p_10,2,5,0.1,1,Imp-then-Reg 4 - linear,0.593441,0.53181,Any[0.5934413009795707],Any[0.5318097996975277],Any[],1.13563,"Dict{Symbol, Any}(:alpha => 0.9, :regtype => :lasso)",0.577878,Imp-then-Reg,syn_X_mar,syn_Y_linear,1 - Lin-MAR
5,n_1000_p_10,2,5,0.1,1,Imp-then-Reg 5 - linear,0.560968,0.497329,Any[0.5609675224934351],Any[0.49732897796408093],Any[],1.45368,"Dict{Symbol, Any}(:alpha => 1.0, :regtype => :lasso)",0.554987,Imp-then-Reg,syn_X_mar,syn_Y_linear,1 - Lin-MAR
6,n_1000_p_10,2,5,0.1,1,Imp-then-Reg 4 - tree,0.58702,0.479833,Any[0.5870200577625153],Any[0.4798332977933134],Any[],1.17518,Dict(:maxdepth => 4),0.547871,Imp-then-Reg,syn_X_mar,syn_Y_linear,1 - Lin-MAR
7,n_1000_p_10,2,5,0.1,1,Imp-then-Reg 5 - tree,0.57186,0.472435,Any[0.5718596427687634],Any[0.47243515467020425],Any[],0.0483472,Dict(:maxdepth => 4),0.537809,Imp-then-Reg,syn_X_mar,syn_Y_linear,1 - Lin-MAR
8,n_1000_p_10,2,5,0.1,1,Imp-then-Reg 4 - rf,0.626915,0.469273,Any[0.6269145092191312],Any[0.4692725673834949],Any[],26.6316,"Dict(:ntrees => 50, :maxdepth => 10)",0.514037,Imp-then-Reg,syn_X_mar,syn_Y_linear,1 - Lin-MAR
9,n_1000_p_10,2,5,0.1,1,Imp-then-Reg 5 - rf,0.552731,0.475783,Any[0.552731214936813],Any[0.47578277074979725],Any[],16.6645,"Dict(:ntrees => 50, :maxdepth => 5)",0.515967,Imp-then-Reg,syn_X_mar,syn_Y_linear,1 - Lin-MAR


In [40]:
combine(groupby(df, [:dataset, :method, :setting]), nrow)

Row,dataset,method,setting,nrow
,String15,String31,String,Int64
1,n_1000_p_10,XGBoost,1 - Lin-MAR,5
2,n_1000_p_10,Oracle X,1 - Lin-MAR,5
3,n_1000_p_10,Oracle XM,1 - Lin-MAR,5
4,n_1000_p_10,Complete Features,1 - Lin-MAR,5
5,n_1000_p_10,CART MIA,1 - Lin-MAR,5
6,n_1000_p_10,RF MIA,1 - Lin-MAR,5
7,n_1000_p_10,Imp-then-Reg 1 - xgboost,1 - Lin-MAR,5
8,n_1000_p_10,Imp-then-Reg 2 - xgboost,1 - Lin-MAR,5
9,n_1000_p_10,Imp-then-Reg 3 - xgboost,1 - Lin-MAR,5


In [ ]:
unique(combine(groupby(df, [:dataset, :method, :setting]), nrow)[:,:nrow])

In [41]:
filter( t-> t[:nrow] < 90, combine(groupby(res, [:dataset, :method, :setting]), nrow))

ArgumentError: ArgumentError: column name "setting" not found in the data frame; existing most similar names are: "X_setting" and "Y_setting"

In [ ]:
pbdata = unique(filter( t -> t[:nrow] < 90, combine(groupby(df, [:dataset, :method, :setting]), nrow))[:,[:method, :setting, :dataset]])

In [ ]:
aux = filter( t -> t[:dataset] ∈ pbdata && t[:nrow] < 10, combine(groupby(df, [:dataset, :method, :setting, :pMissing]), nrow))
unique(aux[:,[:dataset, :setting, :pMissing, :nrow]])

In [ ]:
filter(t -> t[:pMissing] == 0.1, unique(aux[:,[:dataset, :setting, :pMissing, :nrow]]))

In [ ]:
filter(t -> t[:pMissing] == 0.3, unique(aux[:,[:dataset, :setting, :pMissing, :nrow]]))

In [ ]:
filter(t -> startswith(t[:setting], "5"), unique(aux[:,[:dataset, :setting, :pMissing, :nrow]]))